In [2]:
import pandas as pd
from h3 import h3
import folium
import datetime as dt

In [3]:
#функция создания словаря гексов для отображения на карте из xlsx
def xlsx_to_dict(file):
    df = pd.read_excel(file)
    header = list(df)
    dict_list = [] #список словарей
    for i in range (df[header[1]].count()):
        dict_list.append(dict(zip(header, df.iloc[i])))
    
    return dict(zip(df[header[0]], dict_list)) 

In [4]:
#функция создания словаря гексов для отображения на карте из pandasDF
def pd_to_dict(df):    
    header = list(df)
    dict_list = [] #список словарей
    for i in range (df[header[1]].count()):
        dict_list.append(dict(zip(header, df.iloc[i])))
    
    return dict(zip(df[header[0]], dict_list)) 

In [5]:
def visualize_hexagons(hexagons_dict, color='#bb111166', folium_map=None):
    hexagons = list(hexagons_dict.keys())
    polylines = {}
    lat = []
    lng= []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0] #список линий, которые формируют гекс
        lat.extend(map(lambda v:v[0], polyline)) #все широты в один список, для нахождения средней точки
        lng.extend(map(lambda v:v[1], polyline)) 
        polylines[hex] = polyline

    if folium_map is None:
            m = folium.Map(location=(56.839435,60.6063013), min_zoom=4, zoom_start=10, tiles='cartodbpositron')
    else:    
            m = folium_map 
    for polyline in polylines.items():
            popup_msg = ''
            for i in hexagons_dict[polyline[0]].items():
                popup_msg = popup_msg + '<b>' + str(i[0]) + ': '  + '</b>' + str(i[1]) + '</br>'
            popup_l = folium.map.Popup(html=popup_msg, max_width = 200)    
            my_Polyline = folium.PolyLine(locations = polyline[1],
                                          weight = 3,
                                          color = color,
                                          fill = True,
                                          fill_color = "red",                                          
                                          popup = popup_l, #'id гекса: ' + polyline[0] + '\n\nКол-во жителей: ' + str(hexagons_dict[polyline[0]]['cnt_client']),
                                          tooltip = 'Гекс: ' + str(polyline[0])
                                         )
            m.add_child(my_Polyline)
    return m   

In [6]:
#Данные за сентябрь
f1 = 'files/work_hexes_agg_sent.xlsx'
df1 = pd.read_excel(f1)
df1_dict = pd_to_dict(df1)

m1 = visualize_hexagons(df1_dict)
display(m1)

In [7]:
#Данные за октябрь
f2 = 'files/work_hexes_agg_okt.xlsx'
df2 = pd.read_excel(f2)
df2_dict = pd_to_dict(df2)

m2 = visualize_hexagons(df2_dict)
display(m2)